In [2]:
import pandas as pd # manipulação de dados e data frames
import re # regular expressions usamos para retirar a pontuação da mensagem

## DATA SETS ---------
smsData = pd.read_csv("spam.csv", encoding = "latin-1", usecols=["v1", "v2"])## importa o data set como um data-frame, usa o latin-1 com decoding para ler o ficheiro e seleciona as colunas v1 e v2 como as unicas a ser usadas (o ficheiro tem mais 3 colunas mas vazias) 

## TRANSFORMS
smsData.columns = ["Status", "Mensagem"] #renomeia os hearders das colunas

print("Linhas x Colunas", smsData.shape) #print do tamanho da matriz
smsData.head()  # print das primeiras 5 linhas da mat

Linhas x Colunas (5572, 2)


,Status,Mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
print("Distribuição")
print(smsData['Status'].value_counts()) #Quantidade da ham x spam na tabela
print("")
print("Frequência Relativa")
print(smsData['Status'].value_counts(normalize=True)) #Frequencia relativa

Distribuição
ham     4825
spam     747
Name: Status, dtype: int64

Frequência Relativa
ham     0.865937
spam    0.134063
Name: Status, dtype: float64


In [4]:
#Divisão em Sets
randomSample = smsData.sample(frac=1, random_state=666) # Faz suffle da tabela - frac é a fração dos dados a usar (0,0 a 1,0) - random-state = (int) é a seed que o sistema vai usar como "fator de entropia" para gerar o "random"

# Definir o percentagem de dados a usar (0 a 1)
sampleSize = round(len(randomSample) * 0.8)

# Criação do grupo de teste e treino
trainGroup = randomSample[:sampleSize].reset_index(drop=True) #Faz splice dos dados com base no <inserir nome> que serve como indicador da percentagem de dados a usar. neste caso os primeiros 80% da tabela - reset_index faz reset ao indice da tabela
test_set = randomSample[sampleSize:].reset_index(drop=True) #Faz splice dos dados com base na variável "sampleSize" que serve como indicador da percentagem de dados a usar. neste caso os dados que sobram após "ler" 80% da tabela - reset_index faz reset ao indice da tabela


print("Dados de Treino")
print(trainGroup['Status'].value_counts())
print("")
print(trainGroup['Status'].value_counts(normalize = True))

print("")

print("Dados de Teste")
print(test_set['Status'].value_counts())
print("")
print(test_set['Status'].value_counts(normalize = True))

Dados de Treino
ham     3853
spam     605
Name: Status, dtype: int64

ham     0.864289
spam    0.135711
Name: Status, dtype: float64

Dados de Teste
ham     972
spam    142
Name: Status, dtype: int64

ham     0.872531
spam    0.127469
Name: Status, dtype: float64


In [5]:
#Limpeza dos dados
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.replace('\W', ' ') # Limpa a pontuação
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.lower()  #Transforma tudo em letra pequena
trainGroup.head()

,Status,Mensagem
0,ham,lol no ouch but wish i d stayed out a bit longer
1,ham,well done and luv ya all
2,ham,me 2 babe i feel the same lets just 4get abou...
3,ham,gal n boy walking in d park gal can i hold ur...
4,ham,sorry man my stash ran dry last night and i c...


In [6]:
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.split() 

#Transformamos cada mensagem na coluna "Mensagem" numa lista de palavras - Usamos o split para dividir a frase em várias palavras com o espaço como identificador do fim de cada palavra.

palavras = [] 
for mensagem in trainGroup['Mensagem']:
   for word in mensagem:
      palavras.append(word)

palavras = list(set(palavras))

# Com os nested fors vamos adicionando à lista "palavras" cada palavra presente nas várias mensagens. Depois usamos o set para remover todas as palavras duplicadas da lista e finalmente convertemos o objecto de volta a uma lista com o cast do list porque no for são convertidas em string.

print("Total de palavras: ", len(palavras))

Total de palavras:  7819


In [7]:
wordCounter = {}
wordCounter = {unique_word: [0] * len(trainGroup['Mensagem']) for unique_word in palavras}

for index, mensagem in enumerate(trainGroup['Mensagem']):
   for word in mensagem:
      wordCounter[word][index] += 1

wordTable = pd.DataFrame(wordCounter)
wordTable.head()

# Criamos um dicionário em que cada chave é uma palavra da lista "palavra". Depois o for vai popular a tabela com as várias palavras (colunas) e a frequencia delas em cada um dos elementos do CSV (linhas). Na tabela abaixo só aparecem as primeiras 5 linhas porque o default do .head() é 5


,acting,formatting,okors,txt,coping,mtnl,emigrated,08719180248,mecause,skateboarding,...,konw,icon,oooooh,natural,numbers,smsing,tirupur,sad,returned,friendsare
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
trainGroup_processed = pd.concat([trainGroup, wordTable], axis=1)
trainGroup_processed.head()

# Juntamos a "tabela" do trainGroup com a da contagem de palavras para ficarmos com um data frame bonito e limpo que podemos usar para trainar o nosso modelo.

,Status,Mensagem,acting,formatting,okors,txt,coping,mtnl,emigrated,08719180248,...,konw,icon,oooooh,natural,numbers,smsing,tirupur,sad,returned,friendsare
0,ham,"[lol, no, ouch, but, wish, i, d, stayed, out, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[well, done, and, luv, ya, all]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[me, 2, babe, i, feel, the, same, lets, just, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[gal, n, boy, walking, in, d, park, gal, can, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[sorry, man, my, stash, ran, dry, last, night,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Splice 2. Electric Splicealoo - Separamos as mensagens em 2 grupos - ham e spam - para podermos realizar o calculo
ham_messages = trainGroup_processed[trainGroup_processed['Status'] == 'ham']
spam_messages = trainGroup_processed[trainGroup_processed['Status'] == 'spam'] 


# Probabilidades
p_spam = len(spam_messages) / len(trainGroup_processed)
p_ham = len(ham_messages) / len(trainGroup_processed)

#n do Spam é igual à soma todas as palavras de todas as mensagens Spam
totalWordsPerSpam = spam_messages['Mensagem'].apply(len)
n_spam = totalWordsPerSpam.sum()

#n do Ham é igual à soma todas as palavras de todas as mensagens Ham
totalWordsPerHam = ham_messages['Mensagem'].apply(len)
n_ham = totalWordsPerHam.sum()

# Auto explicativo
n_vocabulary = len(palavras)

# Suavização de laplace - Essencialmente ajuda a suavizar erros na aplicação da formula de naive bayes quando uma das probabilidades é 0 (https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece)
alpha = 1

In [10]:
palavra_spam = {unique_word:0 for unique_word in palavras}
palavra_ham = {unique_word:0 for unique_word in palavras}


for word in palavras:
   n_word_given_spam = spam_messages[word].sum()
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   palavra_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() 
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   palavra_ham[word] = p_word_given_ham

#Este é o core do algoritmo, aqui calculamos a probabilidade da cada palavra ser parte de uma mensagem spam ou ham com base do set de palavras de treino que temos no sistema.

In [18]:
def classificador(message):
   message = re.sub('\W', ' ', message)
   message = message.lower().split()
   #Limpamos o texto fornecido pelo user

#V Calculados as odds das cada palavra na mensagem compor um spam/ham. Comparamos as odds e decidimos a qual dos 3 tipos de mensagem pertence o mail
 
   p_spamInput = p_spam
   p_hamInput = p_ham

   for word in message:
      if word in palavra_spam:
         p_spamInput *= palavra_spam[word]

      if word in palavra_ham:
         p_hamInput *= palavra_ham[word]

   print('Odd Spam:', p_spamInput)
   print('Odd Ham:', p_hamInput)

   if p_hamInput > p_spamInput:
      return 'These are not the spams you are looking for'
   elif p_spamInput > p_hamInput:
      return 'Stop Rebel Spam!'
   else:
      return 'Congratulations! You are being analysed! Please do not resist.'

In [24]:
def tester(message):
 # Esta função serve apenas para testar porque a outra ia fazer print das odds para cada um dos mails do csv e meter a classificação comprida do print no data frame 

   message = re.sub('\W', ' ', message)
   message = message.lower().split()
   #Limpamos o texto fornecido pelo user

   p_spamInput = p_spam
   p_hamInput = p_ham

   for word in message:
      if word in palavra_spam:
         p_spamInput *= palavra_spam[word]

      if word in palavra_ham:
         p_hamInput *= palavra_ham[word]

   if p_hamInput > p_spamInput:
      return 'ham'
   elif p_spamInput > p_hamInput:
      return 'spam'
   else:
      return 'unknown'

In [25]:
test_set['Calculado'] = test_set['Mensagem'].apply(tester) #Aplica o teste ao grupo de forma a gerar a coluna "Calculado" e adiciona a mesma ao data frame. Aqui podiamos aplicar a função "classificador" mas o print ia ficar feio.
test_set.head()

,Status,Mensagem,Calculado
0,ham,"Nothing, i got msg frm tht unknown no..",ham
1,ham,Yup it's at paragon... I havent decided whethe...,ham
2,ham,Do u still have plumbers tape and a wrench we ...,ham
3,ham,"Babe, I'm answering you, can't you see me ? Ma...",ham
4,spam,"ou are guaranteed the latest Nokia Phone, a 40...",spam


In [35]:
#Verificar a fiabilidade do algoritmo - Itera cada linha do grupo de testes, compara a classificação pré-existente com a do algoritmo e devolve os resultados. Também faz print das mensagens que falhou em classificar para que seja possível perceber os erros.
certo = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Status'] == row['Calculado']:
      certo += 1
   else:
      print(row['Status'], "|",row['Mensagem'], "|", row['Calculado'])

print("\nResultados")
print('Corretos:', certo)
print('Incorrectos:', total - certo)
print('Ratio:', (certo/total)*100)

ham | No calls..messages..missed calls | spam
spam | thesmszone.com lets you send free anonymous and masked messages..im sending this message from there..do you see the potential for abuse??? | ham
ham | Nokia phone is lovly.. | spam
spam | Latest News! Police station toilet stolen, cops have nothing to go on! | ham
spam | Hello. We need some posh birds and chaps to user trial prods for champneys. Can i put you down? I need your address and dob asap. Ta r | ham
ham | No calls..messages..missed calls | spam
spam | FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv | ham
spam | Guess who am I?This is the first time I created a web page WWW.ASJESUS.COM read all I wrote. I'm waiting for your opinions. I want to be your friend 1/1 | ham
spam | Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net

In [31]:
classificador('Wow so much win! Big Prize. Free Cash!')

Odd Spam: 1.8497556882635062e-26
Odd Ham: 6.485257249309859e-29


'Stop Rebel Spam!'

In [30]:
classificador("Then we'll keep the meeting scheduled for friday")

Odd Spam: 2.1537192782725765e-28
Odd Ham: 3.296142157469712e-22


'These are not the spams you are looking for'

In [36]:
classificador("OMG you're the winner. Lucky prize Tao Bao")

Odd Spam: 5.044308548608223e-27
Odd Ham: 3.5798508608331393e-29


'Stop Rebel Spam!'